# For-Debugging Notebook

This notebook is a notebook for debugging `unlearning` function, based on the following notebook:
- https://www.kaggle.com/code/eleni30fillou/run-unlearn-finetune

## How to Use

1. implement your `unlearning` function;
2. turn on `internet on` in the right panel;
3. set the variable `USE_MOCK` to `True` in the 2nd code cell;
4. (Optional) modity other parameters in the same cell like `n_checkpoints`;
5. if your codes work,
   - turn off `internet on` in the right panel;
   - set the variable `USE_MOCK` to `False` in the 2nd code cell;
   - save the notebook;
   - and submit!

## Updates
- Ver.5:
  - add a stopwatch decorator
  - make `unlearning` return a updated model
- Ver.4: fix seed

In [1]:
import os
import subprocess

import pandas as pd
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18
from torch.utils.data import DataLoader, Dataset

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu' 
DEVICE

'cuda'

In [2]:
torch.manual_seed(128)

Gr = torch.Generator()
Gr.manual_seed(256)

Gf = torch.Generator()
Gf.manual_seed(512)

Gv = torch.Generator()
Gv.manual_seed(1024)

In [3]:
# Mock setting

import logging
import requests
import tqdm
from torch.utils.data import Subset
from torchvision import transforms

USE_MOCK: bool = True

if USE_MOCK:
    logging.warning('Running with Mock')
    logging.warning('In this mode, internet access may be required.')

    # The number of checkpoints in this mode.
    # NOTE: 512 checkpoints are required in this competition.
    n_checkpoints = 5
    
    # The directory for a dataset and a pretrained model
    mock_dir = './mock'
    mock_model_path = os.path.join(mock_dir, "weights_resnet18_cifar10.pth")
    os.makedirs(mock_dir, exist_ok=True)

In [4]:
# It's really important to add an accelerator to your notebook, as otherwise the submission will fail.
# We recomment using the P100 GPU rather than T4 as it's faster and will increase the chances of passing the time cut-off threshold.

if DEVICE != 'cuda':
    raise RuntimeError('Make sure you have added an accelerator to your notebook; the submission will fail otherwise!')

In [6]:
# Helper functions for loading the hidden dataset.

if USE_MOCK:
    
    class DatasetWrapper(Dataset):
        
        def __init__(self, ds: Dataset):
            self._ds = ds
    
        def __len__(self):
            return len(self._ds)
    
        def __getitem__(self, index):
            item = self._ds[index]
            result = {
                'image': item[0],
                'image_id': index,
                'age_group': item[1],
                'age': item[1],
                'person_id': index,
            }
            return result
    
    def get_dataset(batch_size, retain_ratio=0.8, thinning_param: int=1000, root=mock_dir) -> tuple[DataLoader, DataLoader, DataLoader]:
        
        # utils
        normalize = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])

        # create dataset
        train_ds = DatasetWrapper(torchvision.datasets.CIFAR10(root=mock_dir, train=True, download=True, transform=normalize))
        retain_ds = Subset(train_ds, range(0, int(len(train_ds)*retain_ratio), thinning_param))
        forget_ds = Subset(train_ds, range(int(len(train_ds)*retain_ratio), len(train_ds), thinning_param))
        val_ds = DatasetWrapper(torchvision.datasets.CIFAR10(root=mock_dir, train=False, download=True, transform=normalize))

        retain_loader = DataLoader(retain_ds, batch_size=batch_size, shuffle=True)
        forget_loader = DataLoader(forget_ds, batch_size=batch_size, shuffle=True)
        validation_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=True)

        return retain_loader, forget_loader, validation_loader
    
    # For test
#     for sample in get_dataset(3)[0]:
#         print(sample)
#         break
    
else:
    def load_example(df_row):
        image = torchvision.io.read_image(df_row['image_path'])
        result = {
            'image': image,
            'image_id': df_row['image_id'],
            'age_group': df_row['age_group'],
            'age': df_row['age'],
            'person_id': df_row['person_id']
        }
        return result


    class HiddenDataset(Dataset):
        '''The hidden dataset.'''
        def __init__(self, split='train'):
            super().__init__()
            self.examples = []

            df = pd.read_csv(f'/kaggle/input/neurips-2023-machine-unlearning/{split}.csv')
            df['image_path'] = df['image_id'].apply(lambda x: os.path.join('/kaggle/input/neurips-2023-machine-unlearning/', 'images', x.split('-')[0], x.split('-')[1] + '.png'))
            df = df.sort_values(by='image_path')
            df.apply(lambda row: self.examples.append(load_example(row)), axis=1)
            if len(self.examples) == 0:
                raise ValueError('No examples.')

        def __len__(self):
            return len(self.examples)

        def __getitem__(self, idx):
            example = self.examples[idx]
            image = example['image']
            image = image.to(torch.float32)
            example['image'] = image
            return example


    def get_dataset(batch_size):
        '''Get the dataset.'''
        retain_ds = HiddenDataset(split='retain')
        forget_ds = HiddenDataset(split='forget')
        val_ds = HiddenDataset(split='validation')

        retain_loader = DataLoader(retain_ds, batch_size=batch_size, shuffle=True, generator=Gr)
        forget_loader = DataLoader(forget_ds, batch_size=batch_size, shuffle=True, generator=Gf)
        validation_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=True, generator=Gv)

        return retain_loader, forget_loader, validation_loader

In [7]:
# Utils
from contextlib import contextmanager
import time

@contextmanager
def stopwatch(name='STOPWATCH'):
    s = time.time()
    try:
        yield
    finally:
        print(f"{name}: {time.time()-s} seconds passed")
        
# for test
# with stopwatch():
#     for i in range(5):
#         time.sleep(1)

In [10]:
import copy
import itertools
from tqdm import tqdm

def get_mean_var(p, is_base_dist=False, alpha=3e-6):
        
    var = copy.deepcopy(1./(p.grad2_acc+1e-8))
    
    var = var.clamp(max=1e3)
    
    #if p.size(0) == num_classes:
    #    var = var.clamp(max=1e2)
    
    var = alpha * var

    if p.ndim > 1:
        var = var.mean(dim=1, keepdim=True).expand_as(p).clone()
        
    if not is_base_dist:
        mu = copy.deepcopy(p.data0.clone())
    else:
        mu = copy.deepcopy(p.data0.clone())
        
    #if p.size(0) == num_classes and num_to_forget is None:
    #    mu[class_to_forget] = 0
    #    var[class_to_forget] = 0.0001
    #if p.size(0) == num_classes:
        # Last layer
    #    var *= 10
    if p.ndim == 1:
        # BatchNorm
        var *= 10
        var*=1
    return mu, var


def unlearning(
    net, 
    retain_loader, 
    forget_loader, 
    val_loader):

    # netf = copy.deepcopy(net)
    for p in itertools.chain(net.parameters()):
        p.data0 = copy.deepcopy(p.data.clone())
    
    net.eval()
    loss_fn = nn.CrossEntropyLoss()

    for p in net.parameters():
        p.grad_acc = 0
        p.grad2_acc = 0
        
    
    for sample in tqdm(retain_loader):

        data = sample["image"]
        orig_target = sample["age_group"]
        print(orig_target)
        
        data, orig_target = data.to(DEVICE), orig_target.to(DEVICE)
        
        output = net(data)
        print(output)
        
        prob = torch.nn.functional.softmax(output, dim=-1).data
        print(prob)
        
        print(output.shape[1])

        for y in range(output.shape[1]):
            
            target = torch.empty_like(orig_target).fill_(y)
            print(target)
            
            loss = loss_fn(output, target)
            print(loss)
            
            net.zero_grad()
            
            loss.backward(retain_graph=True)
            
            for p in net.parameters():
                if p.requires_grad:
                    p.grad_acc += (orig_target == target).float() * p.grad.data
                    p.grad2_acc += prob[:, y] * p.grad.data.pow(2)
                    
    for p in net.parameters():
        
        p.grad_acc /= len(retain_loader)
        p.grad2_acc /= len(retain_loader)
        print(p.grad_acc.dim)
        print(p.grad2_acc.dim)
        
    alpha = 1e-6
    torch.manual_seed(1756)
    for i, p in enumerate(net.parameters()):
        mu, var = get_mean_var(p, False, alpha=alpha)
        p.data = mu + var* torch.empty_like(p.data0).normal_()
    
    net.eval()
    return net

In [ ]:
import copy
import itertools
from tqdm import tqdm


def unlearning(
    net, 
    retain_loader, 
    forget_loader, 
    val_loader):

    for p in itertools.chain(net.parameters()):
        p.data0 = copy.deepcopy(p.data.clone())

    loss_fn = nn.CrossEntropyLoss()

    for p in net.parameters():
        p.grad_acc = 0
        p.grad2_acc = 0
        
    
    for sample in tqdm(retain_loader):

        data = sample["image"]
        orig_target = sample["age_group"]

        data, orig_target = data.to(DEVICE), orig_target.to(DEVICE)
        output = net(data)
        prob = torch.nn.functional.softmax(output, dim=-1).data

        for y in range(output.shape[1]):
            
            target = torch.empty_like(orig_target).fill_(y)
            print(target)
            
            loss = loss_fn(output, target)
            print(loss)
            
            net.zero_grad()
            
            loss.backward(retain_graph=True)
            
            for p in net.parameters():
                if p.requires_grad:
                    p.grad_acc += (orig_target == target).float() * p.grad.data
                    p.grad2_acc += prob[:, y] * p.grad.data.pow(2)
                    
    for p in net.parameters():
        
        p.grad_acc /= len(retain_loader)
        p.grad2_acc /= len(retain_loader)
        print(p.grad_acc.dim)
        print(p.grad2_acc.dim)
        
    alpha = 1e-6
    torch.manual_seed(1756)
    for i, p in enumerate(net.parameters()):
        mu, var = get_mean_var(p, False, alpha=alpha)
        p.data = mu + var* grad_acc  * torch.empty_like(p.data0).normal_()
    
    net.eval()
    return net

In [11]:
if USE_MOCK:
    
    # NOTE: Almost same as the original codes
    
    # Download
    if not os.path.exists(mock_model_path):
        response = requests.get("https://storage.googleapis.com/unlearning-challenge/weights_resnet18_cifar10.pth")
        open(mock_model_path, "wb").write(response.content)    
    
    os.makedirs('/kaggle/tmp2', exist_ok=True)
    retain_loader, forget_loader, validation_loader = get_dataset(1)
    net = resnet18(weights=None, num_classes=10)
    
    #for batch in retain_loader:
        # Print the type and structure of the batch
    #    print(type(batch))
    #    print(len(batch))  # To see how many values are in the tuple

        # Uncomment the following line to print the batch itself
    #    print(batch)

    #loader_length = len(validation_loader)
    #print("Number of values returned by train_loader:", loader_length)
    #loader_length = len(forget_loader)
    #print("Number of values returned by forget_loader:", loader_length)
    net.to(DEVICE)
    for i in range(n_checkpoints):
        net.load_state_dict(torch.load(mock_model_path))
        net_ = unlearning(net, retain_loader, forget_loader, validation_loader)
        state = net_.state_dict()
        torch.save(state, f'/kaggle/tmp2/unlearned_checkpoint_{i}.pth')

    # Ensure that submission.zip will contain exactly 512 checkpoints 
    # (if this is not the case, an exception will be thrown).
    unlearned_ckpts = os.listdir('/kaggle/tmp2')
    if len(unlearned_ckpts) != n_checkpoints:
        raise RuntimeError('Expected exactly 512 checkpoints. The submission will throw an exception otherwise.')
    
        
    print(os.listdir('/kaggle/tmp2'))
    #subprocess.run('zip submission.zip /kaggle/tmp2/*.pth', shell=True)
    subprocess.run('zip submission.zip /kaggle/tmp2/*.pth', shell=True)
    
else:
    if os.path.exists('/kaggle/input/neurips-2023-machine-unlearning/empty.txt'):
        # mock submission
        subprocess.run('touch submission.zip', shell=True)
    else:

        # Note: it's really important to create the unlearned checkpoints outside of the working directory 
        # as otherwise this notebook may fail due to running out of disk space.
        # The below code saves them in /kaggle/tmp to avoid that issue.

        os.makedirs('/kaggle/tmp', exist_ok=True)
        retain_loader, forget_loader, validation_loader = get_dataset(64)
        net = resnet18(weights=None, num_classes=10)
        net.to(DEVICE)
        for i in range(512):
            net.load_state_dict(torch.load('/kaggle/input/neurips-2023-machine-unlearning/original_model.pth'))
            net_ = unlearning(net, retain_loader, forget_loader, validation_loader)
            state = net_.state_dict()
            torch.save(state, f'/kaggle/tmp/unlearned_checkpoint_{i}.pth')

        # Ensure that submission.zip will contain exactly 512 checkpoints 
        # (if this is not the case, an exception will be thrown).
        unlearned_ckpts = os.listdir('/kaggle/tmp')
        if len(unlearned_ckpts) != 512:
            raise RuntimeError('Expected exactly 512 checkpoints. The submission will throw an exception otherwise.')

        subprocess.run('zip submission.zip /kaggle/tmp/*.pth', shell=True)


Files already downloaded and verified
Files already downloaded and verified


  2%|▎         | 1/40 [00:00<00:04,  9.68it/s]

tensor([8])
tensor([[-0.1813, -1.9498,  0.6527, -0.8094, -3.3110, -0.5479, -2.3913, -1.2656,
         10.9227, -1.1194]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[1.5050e-05, 2.5676e-06, 3.4656e-05, 8.0316e-06, 6.5822e-07, 1.0432e-05,
         1.6510e-06, 5.0894e-06, 9.9992e-01, 5.8908e-06]], device='cuda:0')
10
tensor([0], device='cuda:0')
tensor(11.1041, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([1], device='cuda:0')
tensor(12.8725, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([2], device='cuda:0')
tensor(10.2700, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([3], device='cuda:0')
tensor(11.7321, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([4], device='cuda:0')
tensor(14.2337, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([5], device='cuda:0')
tensor(11.4706, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([6], device='cuda:0')
tensor(13.3141, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([7], device='cuda:0')
tensor(12.1883, 

 10%|█         | 4/40 [00:00<00:03,  9.85it/s]

tensor([1])
tensor([[-0.8671,  8.4216, -1.7000, -1.5789, -2.7154, -0.6648, -0.3491, -0.8559,
          0.1213,  0.1882]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[9.2369e-05, 9.9893e-01, 4.0160e-05, 4.5331e-05, 1.4548e-05, 1.1308e-04,
         1.5505e-04, 9.3414e-05, 2.4819e-04, 2.6536e-04]], device='cuda:0')
10
tensor([0], device='cuda:0')
tensor(9.2897, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([1], device='cuda:0')
tensor(0.0011, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([2], device='cuda:0')
tensor(10.1226, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([3], device='cuda:0')
tensor(10.0015, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([4], device='cuda:0')
tensor(11.1381, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([5], device='cuda:0')
tensor(9.0874, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([6], device='cuda:0')
tensor(8.7718, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([7], device='cuda:0')
tensor(9.2785, devic

...

 92%|█████████▎| 37/40 [00:03<00:00, 10.07it/s]

tensor([[ 9.8667, -2.6076,  2.2265,  1.3939, -0.4768, -1.7761, -3.1672, -2.5319,
         -0.3874, -2.5405]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[9.9922e-01, 3.8207e-06, 4.8033e-04, 2.0891e-04, 3.2175e-05, 8.7750e-06,
         2.1832e-06, 4.1212e-06, 3.5185e-05, 4.0859e-06]], device='cuda:0')
10
tensor([0], device='cuda:0')
tensor(0.0008, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([1], device='cuda:0')
tensor(12.4751, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([2], device='cuda:0')
tensor(7.6410, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([3], device='cuda:0')
tensor(8.4736, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([4], device='cuda:0')
tensor(10.3443, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([5], device='cuda:0')
tensor(11.6436, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([6], device='cuda:0')
tensor(13.0347, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([7], device='cuda:0')
tensor(12.3994, device='cuda:0'

100%|██████████| 40/40 [00:04<00:00,  9.95it/s]

tensor([9], device='cuda:0')
tensor(6.5900, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([5])
tensor([[ 2.9645, -3.6526, -2.7475,  5.8789, -1.7009, 10.7838, -2.4197, -2.1822,
         -4.5472, -2.3774]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[3.9878e-04, 5.3330e-07, 1.3183e-06, 7.3527e-03, 3.7545e-06, 9.9224e-01,
         1.8299e-06, 2.3203e-06, 2.1798e-07, 1.9089e-06]], device='cuda:0')
10
tensor([0], device='cuda:0')
tensor(7.8271, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([1], device='cuda:0')
tensor(14.4442, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([2], device='cuda:0')
tensor(13.5391, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([3], device='cuda:0')
tensor(4.9127, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([4], device='cuda:0')
tensor(12.4925, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([5], device='cuda:0')
tensor(0.0078, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor([6], device='cuda:0')
tensor(13.2113, devi

['unlearned_checkpoint_0.pth', 'unlearned_checkpoint_2.pth', 'unlearned_checkpoint_1.pth', 'unlearned_checkpoint_3.pth', 'unlearned_checkpoint_4.pth']
  adding: kaggle/tmp2/unlearned_checkpoint_0.pth (deflated 7%)
  adding: kaggle/tmp2/unlearned_checkpoint_1.pth (deflated 7%)
  adding: kaggle/tmp2/unlearned_checkpoint_2.pth (deflated 7%)
  adding: kaggle/tmp2/unlearned_checkpoint_3.pth (deflated 7%)
  adding: kaggle/tmp2/unlearned_checkpoint_4.pth (deflated 7%)
